In [4]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit import execute
from qiskit.compiler import assemble
from qiskit import Aer
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from numpy.random import randint
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy import pi


# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")
# Simulate the quantum circuit and get the counts
simulator = Aer.get_backend('qasm_simulator')
# Invoke a primitive. For more details see https://docs.quantum-computing.ibm.com/run/primitives
# result = Sampler().run(circuits).result()

In [5]:
#Alice encodes using phase encoding
def phase_encoding(bit1, bit2, circuit, qreg_q):
    encoding = []
    if bit1==1 and bit2==0: #message is 10, i.e, phase difference is pi and 0
        circuit.p(0, qreg_q[0])
        circuit.p(pi, qreg_q[1])
        circuit.p(pi, qreg_q[2])
        encoding.append('pi 0')
    elif bit1==1 and bit2==1:  #message is 11, i.e, phase difference is pi and pi
        circuit.p(pi, qreg_q[0])
        circuit.p(0, qreg_q[1])
        circuit.p(pi, qreg_q[2])
        encoding.append('pi pi')
    elif bit1==0 and bit2==1:  #message is 01, i.e, phase difference is pi and 0
        circuit.p(0, qreg_q[0])
        circuit.p(0, qreg_q[1])
        circuit.p(pi, qreg_q[2])
        encoding.append('0 pi')
    elif bit1==0 and bit2==0:
        circuit.p(0, qreg_q[0])
        circuit.p(0, qreg_q[1])
        circuit.p(0, qreg_q[2])
        encoding.append('0 0')
    else:
        raise ValueError("Invalid input bits. Please provide valid values (0 or 1) for bit1 and bit2.")
    return circuit, encoding

In [6]:
def alice(bit1, bit2, circuit, qreg_q):
    #create a weak coherent pulse |psi> = 1/sqrt(3)*(|100>+|010>+|001>)
    circuit.x(qreg_q[0])
    circuit.cu(1.91, 0, 0, 0, qreg_q[0], qreg_q[1])
    circuit.cx(qreg_q[1], qreg_q[0])
    circuit.ch(qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    #circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
    
    #phase encoding    
    circuit, encoding = phase_encoding(bit1, bit2, circuit, qreg_q)
    #circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])

    # display the circuit
    #print("Circuit Created by Alice:")
    #display(circuit.draw())
    return circuit, encoding

In [7]:
def bob(circuit, qreg_q, creg_c):
    circuit.h(qreg_q[3])
    circuit.h(qreg_q[4])
    circuit.cx(qreg_q[3], qreg_q[0])
    circuit.cx(qreg_q[3], qreg_q[1])
    circuit.cx(qreg_q[4], qreg_q[1])
    circuit.cx(qreg_q[4], qreg_q[2])
    circuit.h(qreg_q[3])
    circuit.h(qreg_q[4])
    
    #circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
    circuit.measure(qreg_q[3], creg_c[3])
    circuit.measure(qreg_q[4], creg_c[4])
    
    
     # Get the classical bit values from the result
    result = execute(circuit, Aer.get_backend('qasm_simulator'), shots=1000).result()
    counts = result.get_counts(circuit)
    
    print(f"Intermediate results: {counts}")
    c3_value  = int(list(counts.keys())[3], 2)
    c4_value  = int(list(counts.keys())[4], 2)

    #c3_value = int(result.get_counts(circuit).most_frequent()[3])
    #c4_value = int(result.get_counts(circuit).most_frequent()[4])
    return counts, c3_value, c4_value

In [8]:
def dps_qkd(alice_key):
    #create arrays to store phase difference and key
    encoding_array = []
    bob_key = []
    #we create a circuit with 5 qubits: 3 in use, 2 ancillary
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')

     # Create a quantum circuit with the desired registers
    circuit = QuantumCircuit(qreg_q, creg_c)
    
    # Initialize all qubits to the |0⟩ state
    circuit.initialize([1] + [0] * (2 ** qreg_q.size - 1), qreg_q)
    
    # Check if 'alice_key' is a NumPy array, and convert if necessary
    if not isinstance(alice_key, np.ndarray):
        alice_key = np.array(alice_key)
        
     # Convert the NumPy array to a Python list
    alice_key = alice_key.tolist()
    #This is a 3 qubit circuit, so we can send 2 bits of phase information at a time. Thus, we break down a message into pairs of two qubits at a time.
    if len(alice_key) % 2 != 0:
        # Add 0 at the end to make the length even
        alice_key.append(0)
    # Use list comprehension to create pairs
    groups_of_2 = [(alice_key[i], alice_key[i + 1]) for i in range(0, len(alice_key), 2)]
    
    # alice and bob exchange 2 bits
    for group in groups_of_2:
        #alice sends 2 bits
        circuit, encoding = alice(*group, circuit, qreg_q)
        encoding_array.append(encoding)
        counts, c3_value, c4_value = bob(circuit, qreg_q, creg_c)
        bob_key.append(c3_value)
        bob_key.append(c4_value)
    return encoding_array, bob_key

In [9]:
# Example usage
alice_key = [0, 1, 1, 0]
encoding_array, bob_key = dps_qkd(alice_key)
print(f"Key sent by Alice: {alice_key}")
print(f"Key received by Bob: {bob_key}")
print(f"Phase encoding used by Alice: {encoding_array}")

Intermediate results: {'11000': 80, '01000': 82, '00000': 80, '10000': 758}
Traceback (most recent call last):
  Cell In[9], line 3
    encoding_array, bob_key = dps_qkd(alice_key)
  Cell In[8], line 33 in dps_qkd
    counts, c3_value, c4_value = bob(circuit, qreg_q, creg_c)
  Cell In[7], line 22 in bob
    c4_value  = int(list(counts.keys())[4], 2)
IndexError: list index out of range

Use %tb to get the full traceback.


In [10]:
# call the function
n=100
np.random.seed(seed=0)
alice_key = randint(2, size=n)
encoding_array, bob_key = dps_qkd(alice_key)
print(f"Key sent by Alice: {alice_key}")
print(f"Key received by Bob: {bob_key}")
print(f"Phase encoding used by Alice: {encoding_array}")

Intermediate results: {'00000': 88, '01000': 100, '11000': 95, '10000': 717}
Traceback (most recent call last):
  Cell In[10], line 5
    encoding_array, bob_key = dps_qkd(alice_key)
  Cell In[8], line 33 in dps_qkd
    counts, c3_value, c4_value = bob(circuit, qreg_q, creg_c)
  Cell In[7], line 22 in bob
    c4_value  = int(list(counts.keys())[4], 2)
IndexError: list index out of range

Use %tb to get the full traceback.


In [8]:
#Block to print state vector

from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Statevector

# Create a quantum circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Define the simulator
simulator = Aer.get_backend('statevector_simulator')

# Simulate the circuit and obtain the state vector of the output state
output_state = execute(qc, simulator).result().get_statevector()

# Create a Statevector object
statevector = Statevector(output_state)

# Print the ket vector notation
ket_vector_notation = statevector.data
print("Output State in Ket Vector Notation:")
print(ket_vector_notation)

Output State in Ket Vector Notation:
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [19]:
from qiskit import QuantumCircuit, Aer, execute, ClassicalRegister, QuantumRegister

# Create a quantum circuit with a classical register
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

# Apply some quantum operations, e.g., a measurement
circuit.measure(qreg_q[0], creg_c[0])

# Simulate the circuit using the statevector simulator
backend = Aer.get_backend('qasm_simulator')
result = execute(circuit, backend).result()

# Get the counts from the result
counts = result.get_counts(circuit)

# Extract the classical bit value from the counts
bit_value = int(list(counts.keys())[0], 2)

# Print the result
print(f"Classical bit value: {bit_value}")


Classical bit value: 0
